In [1]:
from pylibCZIrw import czi as pyczi
import json
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import numpy as np
from numpy import ma
import os, sys
from tqdm import tqdm
from tqdm.contrib import itertools as it
from matplotlib.patches import Rectangle
from matplotlib.colors import LinearSegmentedColormap
from cellpose import utils
from scipy import ndimage
from pathlib import Path
import pandas as pd
from skimage import io, exposure
import cv2
import json
from shapely.geometry import mapping, Polygon
from skimage.filters import threshold_otsu
import shutil
import geopandas as gpd
import rasterio
import rasterio.features
import numpy as np

# show the used python env
print("Using:", sys.executable)

Using: /home/chrism/miniconda3/envs/cellpose3/bin/python


In [ ]:
height = 1040
width = 1388

# nuclei extration
debug = False
border = 2

what = "test"

if what == "test":
    data_path = Path('/home/chrism/datasets/dapidl/test/endothelialcell')
else:
    data_path = Path('/home/chrism/datasets/dapidl/train/endothelialcell')

colors_blue = [(0, 0, 0), (0, 0, 1)]
cm_blue = LinearSegmentedColormap.from_list("Custom", colors_blue, N=255)

colors_red = [(0, 0, 0), (1, 0, 0)]
cm_red = LinearSegmentedColormap.from_list("Custom", colors_red, N=255)

colors_green = [(0, 0, 0), (0, 1, 0)] 
cm_green = LinearSegmentedColormap.from_list("Custom", colors_green, N=255)

colors_white = [(0, 0, 0), (1, 1, 1)] 
cm_white = LinearSegmentedColormap.from_list("Custom", colors_white, N=255)

# for otsu_correction in np.arange(0.7, 1.3, 0.02):
for otsu_correction in [1.0]:
    print(f"Otsu correction: {otsu_correction}")
    area_pixel = 0.512 * 0.512
    nuclei_path = data_path / "nuclei"
    cells_path = data_path / "cells"
    czi_path = data_path / "czi"
    exclude_path = data_path / "exclude"
    debug_path = data_path / "debug_otsu"
    classification_path = data_path / f"classification_otsu_{otsu_correction:.2f}"
    nuclei_tumorcells_path = data_path / f"nuclei_endothelialcells_otsu_{otsu_correction:.2f}"
    results = []
    
    for nuclei_masks_file in nuclei_path.glob("**/*.npy"):
        # Prepare GeoJSON structure
        geojson = {
            "type": "FeatureCollection",
            "features": []
        }
        print(nuclei_masks_file)
        slide = nuclei_masks_file.parent.parts[-1]
        group = nuclei_masks_file.parent.parts[-2]
        
        exclude_masks = exclude_path / (nuclei_masks_file.stem[:-4] + ".geojson")
        if not exclude_masks.exists():
            print(f"Exclude mask {exclude_masks} does not exist. Skipping ...")
            continue
            
        czi_file_name = nuclei_masks_file.name[0:-8] + '.czi'
        czi_file = czi_path / czi_file_name
        if not czi_file.exists():
            print(f"CZI file {czi_file} does not exist. Skipping ...")
            continue
            
        # cells_masks = cells_path / nuclei_masks_file.name
        # if not cells_masks.exists():
        #     print(f"{cells_masks} does not exist. Skipping ...")
        #     continue
        
    
        nuclei_seg = np.load(nuclei_masks_file, allow_pickle=True).item()
        nuclei_masks = nuclei_seg["masks"]
        num_nuclei = nuclei_masks.max()
        
        # exclude_seg = np.load(exclude_masks, allow_pickle=True).item()
        # exclude_masks = exclude_seg["masks"] > 0

        # Load the GeoJSON file
        gdf = gpd.read_file(exclude_masks)

        # Define the dimensions of the mask
        exclude_masks = np.zeros((height, width), dtype=np.uint8)

        # Convert the GeoJSON geometries into a binary mask
        for _, row in gdf.iterrows():
            if row['geometry'] is not None:
                exclude_masks |= rasterio.features.rasterize([(row['geometry'], 1)],
                                                    out_shape=(height, width))
    
        # cells_seg = np.load(cells_masks, allow_pickle=True).item()
        # cells_masks = cells_seg["masks"]
        # num_cells = cells_masks.max()

        num_pixels = exclude_masks.shape[0] * exclude_masks.shape[1] - np.count_nonzero(exclude_masks)
        area = num_pixels * area_pixel
        # print(area)
        nuclei_per_um2 = num_nuclei / area
        # print(cells_per_um2)
    
        with pyczi.open_czi(str(czi_file)) as czidoc:
            ch0 = czidoc.read(plane={'C': 0})
            dapi = ch0[..., 0]
            ch1 = czidoc.read(plane={'C': 1})
            mrc1 = ch1[..., 0]
            # ch2 = czidoc.read(plane={'C': 2})
            # vwf = ch2[..., 0]
            # ch3 = czidoc.read(plane={'C': 3})
            # f480 = ch3[..., 0]
    
        if debug:
            # d_slices = utils.find_objects(masks)
            # d_mrc1 = mrc1.copy()
            # for i,si in enumerate(slices):
            #     if si is not None:
            #         sr,sc = si
            #         mask = (masks[sr, sc] == (i+1)).astype(np.uint8)
            #         contours = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            #         ax.contours(
            #         d_mrc1 = cv2.drawContours(d_mrc1, contours, -1, (0, 255, 0), 2)
                    
            plt.ioff()
            fig, ax = plt.subplots(figsize=(13, 10))
            ax.imshow(mrc1, cmap=cm_red)
    
        # cells_slices = utils.find_objects(cells_masks)
        # cells_centers = ndimage.center_of_mass(mrc1, cells_masks, index=np.arange(1, num_cells + 1))
    
        # for i, (si, (row, col)) in enumerate(zip(cells_slices, cells_centers)): 
        #     if np.any((cells_masks == i+1) & exclude_masks):
        #         print(f"Cell {i+1} intersecting exclude region. Skipping ...")
        #         continue
        #     if si is not None:
        #         sr, sc = si
        #         cell_mask = cells_masks[sr, sc] != (i+1)
        #         dapi_ma = ma.masked_array(dapi[sr, sc], mask)
    
        
        slices = utils.find_objects(nuclei_masks)
        nuclei_centers = ndimage.center_of_mass(dapi, nuclei_masks, index=np.arange(1, num_nuclei + 1))
        
        # pass 1 otsu
        nucleus_means = []
        for i, (si, (row, col)) in enumerate(zip(slices, nuclei_centers)):
            sr, sc = si
            if sr.start - border >= 0:
                sr_start = sr.start - border
            else:
                sr_start = sr.start
            if sr.stop + border < 1040:
                sr_stop = sr.stop + border
            else:
                sr_stop = sr.stop
            if sc.start - border >= 0:
                sc_start = sc.start - border
            else:
                sc_start = sc.start
            if sc.stop + border < 1388:
                sc_stop = sc.stop + border
            else:
                sc_stop = sc.stop
            mask = nuclei_masks[sr_start:sr_stop, sc_start:sc_stop] != (i+1)
            dapi_ma = ma.masked_array(dapi[sr_start:sr_stop, sc_start:sc_stop], mask)
            mrc1_ma = ma.masked_array(mrc1[sr_start:sr_stop, sc_start:sc_stop], mask)
            nucleus_mask = np.ones(nuclei_masks.shape, dtype=bool)
            nucleus_mask[sr_start:sr_stop, sc_start:sc_stop] = mask
            if np.any((nuclei_masks == i+1) & exclude_masks):
                # print(f"Nucleus {i+1} intersecting exclude region. Skipping ...")
                continue
            if si is None:
                print(f"Nucleus {i+1} no si. Skipping ...")
                continue
            
            # mrc1_ma = ma.masked_array(mrc1[sr, sc], mask)
            # vwf_ma = ma.masked_array(vwf[sr, sc], mask)
            # f480_ma = ma.masked_array(f480[sr, sc], mask)
            # mrc1_f480_ratio = mrc1_ma.sum() / f480_ma.sum()
            # f480_mrc1_ratio = f480_ma.sum() / mrc1_ma.sum()
            # print(f"MRC1 F4/80 ratio = {mrc1_f480_ratio}")
            mrc1mean = mrc1_ma.mean()
            nucleus_means.append(mrc1mean)
        deepred_threshold = threshold_otsu(np.array(nucleus_means)) * otsu_correction
        print("Otsu's threshold:", deepred_threshold)
    
        
        # pass 2
        for i, (si, (row, col)) in enumerate(zip(slices, nuclei_centers)): 
            sr, sc = si
            if sr.start - border >= 0:
                sr_start = sr.start - border
            else:
                sr_start = sr.start
            if sr.stop + border < 1040:
                sr_stop = sr.stop + border
            else:
                sr_stop = sr.stop
            if sc.start - border >= 0:
                sc_start = sc.start - border
            else:
                sc_start = sc.start
            if sc.stop + border < 1388:
                sc_stop = sc.stop + border
            else:
                sc_stop = sc.stop
            mask = nuclei_masks[sr_start:sr_stop, sc_start:sc_stop] != (i+1)
            dapi_ma = ma.masked_array(dapi[sr_start:sr_stop, sc_start:sc_stop], mask)
            mrc1_ma = ma.masked_array(mrc1[sr_start:sr_stop, sc_start:sc_stop], mask)
            nucleus_mask = np.ones(nuclei_masks.shape, dtype=bool)
            nucleus_mask[sr_start:sr_stop, sc_start:sc_stop] = mask
            if np.any((nuclei_masks == i+1) & exclude_masks):
                # print(f"Nucleus {i+1} intersecting exclude region. Skipping ...")
                nuclei_masks[np.invert(nucleus_mask)] = 0
                continue
            if si is None:
                print(f"Nucleus {i+1} no si. Skipping ...")
                nuclei_masks[np.invert(nucleus_mask)] = 0
                continue
            
            # mrc1_ma = ma.masked_array(mrc1[sr, sc], mask)
            # vwf_ma = ma.masked_array(vwf[sr, sc], mask)
            # f480_ma = ma.masked_array(f480[sr, sc], mask)
            # mrc1_f480_ratio = mrc1_ma.sum() / f480_ma.sum()
            # f480_mrc1_ratio = f480_ma.sum() / mrc1_ma.sum()
            # print(f"MRC1 F4/80 ratio = {mrc1_f480_ratio}")
            mrc1mean = mrc1_ma.mean()

            if mrc1mean > deepred_threshold:
                nucleus_class = "endothelialcell"
                nuclei_masks[np.invert(nucleus_mask)] = 1
            else:
                nucleus_class = "other"
                nuclei_masks[np.invert(nucleus_mask)] = 2

            _, contours, _ = cv2.findContours((nuclei_masks == i+1).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            for contour in contours:
                # Convert contour to polygon and skip invalid polygons
                if contour.shape[0] < 3:
                    continue
                polygon = Polygon(contour.squeeze())
                if not polygon.is_valid:
                    continue
                # Add polygon to GeoJSON
                feature = {
                    "type": "Feature",
                    "properties": {
                        "label": nucleus_class 
                    },
                    "geometry": mapping(polygon)
                }
                geojson["features"].append(feature)
            # print(row, col)
            row = round(row)
            col = round(col)
            
            # save image
    
            im12 = dapi_ma.filled(0)
            # im8 = exposure.rescale_intensity(im12, in_range=(0, 2**12), out_range=np.uint8)
            im8 = exposure.rescale_intensity(dapi[sr, sc], in_range=(0, 2**12), out_range=np.uint8)
            png_file_name = f"{czi_file_name}_{i:0>6}.png"
            png_file_path = classification_path / nucleus_class / png_file_name
            if not png_file_path.parent.exists():
                png_file_path.parent.mkdir(parents=True, exist_ok=True)
            io.imsave(png_file_path, im8)
            results_row = {'czi': czi_file_name, 'nucleus_no': i, 'class': nucleus_class, 'filename': png_file_name, 'mean(dapi)': dapi_ma.mean(), 'mean(deepred)': mrc1mean, 'area nucleus': mask.sum(), 'pos nucleus': f'{row},{col}', 'deepred threshold': deepred_threshold}
            results.append(results_row)
    
            if debug:
                ax.text(col, row, str(i), color="white")
                ax.scatter(col, row, c="yellow", marker=".")
                # d_mask = (cell_masks[sr, sc] == (i+1)).astype(np.uint8)
                # d_contours, _ = cv2.findContours(d_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
                # d_contours, _ = cv2.findContours(d_mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
                # d_contour = d_contours[0]
                
                # M = cv2.moments(d_contour)
                # x = int(M["m10"] / M["m00"])
                # y = int(M["m01"] / M["m00"])
                
                # xs = [v[0][0] - x for v in d_contour]
                # ys = [-(v[0][1] - y) for v in d_contour]
                
                # ax.plot(xs, ys, color="green")
        
        nuclei_seg["masks"] = nuclei_masks
        if not nuclei_tumorcells_path.exists():
            nuclei_tumorcells_path.mkdir(parents=True, exist_ok=True)
        np.save(nuclei_tumorcells_path / nuclei_masks_file.name, nuclei_seg)
        
        geojson_path = nuclei_tumorcells_path / f"{czi_file_name}.geojson"
        with open(geojson_path, 'w') as f:
            json.dump(geojson, f)
        
        nuclei_image_file_name = nuclei_masks_file.name[:-8] + ".png"
        
        shutil.copy(nuclei_path / nuclei_image_file_name, nuclei_tumorcells_path / nuclei_image_file_name)
        
        if debug:
            debug_file_name = nuclei_masks_file.name + ".png"
            debug_file = debug_path / debug_file_name
            debug_file.parent.mkdir(parents=True, exist_ok=True)
            fig.savefig(str(debug_file))
            plt.close(fig)
            plt.ion()
            
        
    df = pd.DataFrame(results)
    out_fn = f"results_nuclei_border_{border}_otsu_{otsu_correction:.2f}.xlsx"
    df.to_excel(data_path / out_fn)

Otsu correction: 1.0
/home/chrism/datasets/dapidl/test/endothelialcell/nuclei/Ms1760 KO EXT1 Xylt___Snap-161926_seg.npy
Otsu's threshold: 232.2502756503066
